In [1]:
import pandas as pd

import torch
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np
import pickle
from pathlib import Path



In [2]:
df = pd.read_csv("../data/input.csv")
df.head()

,Unnamed: 0,title,img_captions_str,meme_captions_str,metaphors_str,url,input
0,0,For real though,Person in Spider Man outfit gives a lecture on...,Meme poster is frustrated about the format of ...,"Spider Man outfit: Meme poster, a lecture: com...",https://i.redd.it/m16dhaqyply21.jpg,TEXT: For real though; Person in Spider Man ou...
1,1,And that's a fact,"Two dogs carry a white flag, Identical chubby ...",Meme poster is saying that searching Google pl...,"Two dogs: Meme poster, a white flag: PSA, carr...",https://i.redd.it/z9oh7ligb0i31.jpg,TEXT: And that's a fact; Two dogs carry a whit...
2,2,It was horrible,"man is very dissatisfied with what he sees, Qu...",Meme poster is feeling sad.,"man: Meme poster, sees: expressing different r...",https://i.redd.it/yves3izsbsj31.jpg,TEXT: It was horrible; man is very dissatisfie...
3,3,This is why Reddit is better,A man that is crying and a man that is laughin...,Meme poster is saying that Reddit is more fun ...,"A man: Meme poster, a man: Meme poster, his go...",https://i.redd.it/y594n8exi6k31.jpg,TEXT: This is why Reddit is better; A man that...
4,4,The Area 51 raid is still happening right?,"a cop is chasing a thief with a shopping cart,...",Meme poster is making fun of the Area 51 raid.,"a cop: Meme poster, a thief: not being bothere...",https://i.redd.it/4hrn18t4lck31.jpg,TEXT: The Area 51 raid is still happening righ...


In [5]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model = model.to("mps")

In [6]:
embeddings = model.encode(df["input"].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/182 [00:00<?, ?it/s]

In [18]:
with open('meme-embeddings.pkl', "wb") as fOut:
    pickle.dump(embeddings, fOut)

In [8]:
print(f"Shape of one embedding{embeddings[0].shape}")

Shape of one embedding(768,)


In [15]:
from sentence_transformers import util
import pandas as pd
prompt = "Dustbin"
prompt_embedding = model.encode(prompt, convert_to_tensor=True)
hits = util.semantic_search(prompt_embedding,embeddings, top_k=5)
hits = pd.DataFrame(hits[0], columns=['corpus_id', 'score'])

In [16]:
desired_ids = hits["corpus_id"]

In [22]:
filtered_df = df.loc[df['id'].isin(desired_ids)]
list(filtered_df["url"])

['https://i.redd.it/j5ydxwew5o861.jpg',
 'https://i.redd.it/bollrng5030a1.jpg',
 'https://i.redd.it/xtz2v9omoc1a1.jpg',
 'https://i.redd.it/3ijlnsm9el1a1.jpg',
 'https://i.redd.it/lgvmd586lo2a1.jpg']

In [10]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 5818


In [11]:
df.iloc[5415, 1]

5420

In [ ]:
D, I = index.search(np.array([embeddings[5415]]), k=3)
print(f'L2 distance: {D.flatten().tolist()}\n\nMemes: {I.flatten().tolist()}')

In [ ]:
D, I = index.search(np.array([embeddings[5415]]), k=3)